In [126]:
import os
import numpy as np
import pandas as pd
import math as math
import time
from pandas import DataFrame, Series
import datetime
from matplotlib import pyplot as plt
import matplotlib.path as mplPath
%matplotlib inline

In [127]:
# Read in Data
geographic = pd.read_csv("geographic.csv")

uber_trips_2014 = pd.read_csv("uber_trips_2014.csv") 

green_trips = pd.read_csv("green_trips.csv")

yellow_trips_2014Q2 = pd.read_csv("yellow_trips_2014Q2.csv")
yellow_trips_2014Q3 = pd.read_csv("yellow_trips_2014Q3.csv")
yellow_trips_2015Q1 = pd.read_csv("yellow_trips_2015Q1.csv")
yellow_trips_2015Q2 = pd.read_csv("yellow_trips_2015Q2.csv")


KeyboardInterrupt



In [ ]:
# Compute NTA Paths
NTA_names = geographic.loc[[1]]
NTA_paths = []
for x in NTA_names:
    geo_x_filtered = filter(lambda k: not math.isnan(k), geographic[x])
    num_vertices = min(2000, len(geo_x_filtered))
    polygon = map(lambda i: [geo_x_filtered[2*i], geo_x_filtered[2*i+1]], xrange(0, num_vertices/2))
    NTA_paths.append((x, mplPath.Path(np.array(polygon))))

In [ ]:
# Add a NTA column to data
sample = uber_trips_2014.sample(n=50000)
sLength = len(sample['pickup_datetime'])
sample['NTA'] = Series(np.random.randn(sLength))

for (district, path) in NTA_paths:
    latitude = -73.9549
    longitude = 40.769
    for (district, path) in NTA_paths[0:1]:
        if path.contains_point((longitude, latitude)):
            print 'found district'
            break
print 'done'

In [ ]:
start_time = time.time()
districts = []

for index, row in sample[0:50000].iterrows():
    latitude = row['pickup_latitude']
    longitude = row['pickup_longitude']
    
    loc = "None"
    if (index % 1000 == 0):
        print "i = ", index
    for (district, path) in NTA_paths:
        if path.contains_point((longitude, latitude)):
            loc = district
            break
    districts.append(loc)
time_taken = time.time() - start_time
print 'time taken: ', time_taken

In [ ]:
sample["NTA"] = districts
sample["count"] = 1

In [ ]:
sample['pickup_date'] = pd.DatetimeIndex(
    sample['pickup_datetime'].apply(pd.to_datetime)).date

In [ ]:
sample_use = sample[['pickup_date', 'NTA', 'count']]
sample_use.columns = ['pickup_date', 'nta_code', 'count']

use = sample_use[["nta_code", "pickup_date", "count"]].groupby(by=["nta_code", "pickup_date"])
final = use.sum()
final.to_csv("2014_trips_by_NTA_days.csv")

use2 = sample_use[["nta_code", "pickup_date", "count"]].groupby(by=["nta_code"])
final2 = use2.sum()
final2.to_csv("2014_trips_by_NTA_year.csv")

In [ ]:
sampleg = green_trips.sample(n=100000)
sLength = len(sampleg['pickup_datetime'])
sampleg['nta_code'] = Series(np.random.randn(sLength))

start_time = time.time()
districts = []

for index, row in sampleg[0:100000].iterrows():
    latitude = row['pickup_latitude']
    longitude = row['pickup_longitude']
    
    loc = "None"
    if (index % 1000 == 0):
        print "i = ", index
    for (district, path) in NTA_paths:
        if path.contains_point((longitude, latitude)):
            loc = district
            break
    districts.append(loc)
time_taken = time.time() - start_time
print 'time taken: ', time_taken

In [ ]:
sampleg["NTA"] = districts
sampleg["count"] = 1
print "districts done"

sampleg['pickup_date'] = pd.DatetimeIndex(
    sampleg['pickup_datetime'].apply(pd.to_datetime)).date
print "datetime conversion done"

In [ ]:
sampleg_use = sampleg[['pickup_date', 'NTA', 'count']]
sampleg_use.columns = ['pickup_date', 'nta_code', 'count']

use = sampleg_use[["nta_code", "pickup_date", "count"]].groupby(by=["nta_code", "pickup_date"])
final = use.sum()
final.to_csv("green_trips_by_NTA_days.csv")

use2 = sampleg_use[["nta_code", "pickup_date", "count"]].groupby(by=["nta_code"])
final2 = use2.sum()
final2.to_csv("green_trips_by_NTA_year.csv")

In [ ]:
yellow_trips_2014Q2 = pd.read_csv("yellow_trips_2014Q2.csv", nrows=500000)
yellow_trips_2014Q3 = pd.read_csv("yellow_trips_2014Q3.csv", nrows=500000)
yellow_trips_2015Q1 = pd.read_csv("yellow_trips_2015Q1.csv", nrows=500000)
yellow_trips_2015Q2 = pd.read_csv("yellow_trips_2015Q2.csv", nrows=500000)

In [ ]:
sampley1 = yellow_trips_2014Q2.sample(n=25000)
sampley2 = yellow_trips_2014Q3.sample(n=25000)
sampley3 = yellow_trips_2015Q1.sample(n=25000)
sampley4 = yellow_trips_2015Q2.sample(n=25000)

sample2014 = pd.concat([sampley1, sampley2])
sample2015 = pd.concat([sampley3, sampley4])

#########################################################################################################

def find_districts(sampleg, rows):
    sLength = len(sampleg['pickup_datetime'])
    sampleg['nta_code'] = Series(np.random.randn(sLength))

    start_time = time.time()
    districts = []

    for index, row in sampleg[0:rows].iterrows():
        latitude = row['pickup_latitude']
        longitude = row['pickup_longitude']

        loc = "None"
        if (len(districts) % 1000 == 0):
            print "i = ", index
        for (district, path) in NTA_paths:
            if path.contains_point((longitude, latitude)):
                loc = district
                break
        districts.append(loc)
    time_taken = time.time() - start_time
    print 'time taken: ', time_taken
    return districts

#########################################################################################################

districts2014 = find_districts(sample2014, 50000)
districts2015 = find_districts(sample2015, 50000)

In [ ]:
def insertDistricts(sample, districts):
    sample["NTA"] = districts
    sample["count"] = 1
    print "districts done"

    sample['pickup_date'] = pd.DatetimeIndex(
        sample['pickup_datetime'].apply(pd.to_datetime)).date
    print "datetime conversion done"
    return sample

In [ ]:
sample2014 = insertDistricts(sample2014, districts2014)
sample2015 = insertDistricts(sample2015, districts2015)

In [ ]:
def output_stuff(sample, filenames):
    sample_use = sample[['pickup_date', 'NTA', 'count']]
    sample_use.columns = ['pickup_date', 'nta_code', 'count']
    use = sample_use[["nta_code", "pickup_date", "count"]].groupby(by=["nta_code", "pickup_date"])
    final = use.sum()
    final.to_csv(filenames[0])
    
    use2 = sampleg_use[["nta_code", "pickup_date", "count"]].groupby(by=["nta_code"])
    final2 = use2.sum()
    final2.to_csv(filenames[1])
    return 42

output_stuff(sample2014, ["yellow_2014_by_days.csv", "yellow_2014_by_year.csv"])
output_stuff(sample2015, ["yellow_2015_by_days.csv", "yellow_2015_by_year.csv"])